In [1]:
# models
from src.architectures.model_util import load_model, save_checkpoint

# Pytorch Lightning
import pytorch_lightning as pl

# datetime
from datetime import datetime

# other files
from src.run import *
from src.config import *
from src.lit_model import *
from src.data_util import *

In [2]:
cfg = load_config()

In [3]:
today = datetime.today()
fmt = fmt = lambda t: str(t).zfill(2)
today = f'{today.year}.{fmt(today.month)}.{fmt(today.day)}.{fmt(today.hour)}:{fmt(today.minute)}'

loggers = [pl.loggers.TensorBoardLogger("training/logs", name='')]
callbacks = [pl.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=10)]

# cfg['lamb'] = False
model, cfg.device = load_model(cfg.model_name, gpu=False, verbose=False)
lit_model = BaseLitModel(model, cfg=cfg)

Model Resnet is selected.


In [4]:
trainer = pl.Trainer(gpus=1, logger=loggers, callbacks=callbacks)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [5]:
cfg.test = False
trn_dset = MyDataset(cfg, augment=None)
trn_dl = DataLoader(trn_dset, cfg.batch_size)
cfg.test = True
val_dset = MyDataset(cfg, augment=None)
val_dl = DataLoader(val_dset, cfg.batch_size)

In [6]:
trainer.tune(lit_model, train_dataloader=trn_dl, val_dataloaders=val_dl)

In [7]:
trainer.fit(lit_model, train_dataloader=trn_dl, val_dataloaders=val_dl)


  | Name      | Type              | Params
------------------------------------------------
0 | model     | ResNet            | 888 K 
1 | train_acc | MeanAbsoluteError | 0     
2 | val_acc   | MeanAbsoluteError | 0     
3 | test_acc  | MeanAbsoluteError | 0     
------------------------------------------------
888 K     Trainable params
0         Non-trainable params
888 K     Total params


1

In [21]:
dir(lit_model.logger)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_prefix',
 '_agg_default_func',
 '_agg_key_funcs',
 '_aggregate_metrics',
 '_convert_params',
 '_finalize_agg_metrics',
 '_flatten_dict',
 '_logger_iterable',
 '_metrics_to_agg',
 '_prev_step',
 '_reduce_agg_metrics',
 '_sanitize_callable_params',
 '_sanitize_params',
 'agg_and_log_metrics',
 'close',
 'experiment',
 'finalize',
 'log_graph',
 'log_hyperparams',
 'log_metrics',
 'name',
 'save',
 'save_dir',
 'update_agg_funcs',
 'version']

In [14]:
lit_model.val_acc

MeanAbsoluteError()

In [17]:
len(trn_dl)

44